# Perceptron



In [1]:
import numpy as np
np.seterr('ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Perceptron Network

In [2]:
# Here I am building a normal forward propogation for a perceptron layer
# and the various function i would use.

In [3]:
#############################################
## Declaring Training Data        ############
#############################################
X_train = np.array([[0,0],[1,0],[0,1],[1,1]])
Y_train = np.array([[1],[0],[0],[0]])



In [4]:
from MultiLayerPerceptron import MultiLayerPerceptron

In [5]:
# Declare a neuron with shape of weights as [shape_of_input,1]
model = MultiLayerPerceptron([2,1],['sigmoid'])
# print(model.layers[1].W)
# print(model.layers[1].b)

(2, 1) (1,)
<function sigmoid at 0x7fb06d3c82f0>


In [6]:
# testing the model outputs
pred , _ = model.forward(X_train)
# print(np.sum((pred > 0.5)== Y_train) / Y_train.shape[0])
# accuracy(model , X_train,Y_train)
# X_train[0]

In [7]:
# Checking for testing purposes(BCE should be used here ideally)
from Loss import mean_abs_error,mean_square_error
loss,d_back = mean_square_error(pred,Y_train)
print("The loss is {}\nAnd the error to be back propogated is:\n {}".format(loss , d_back))

The loss is [0.25998501]
And the error to be back propogated is:
 [[-0.125     ]
 [ 0.10798757]
 [ 0.22948564]
 [ 0.22370188]]


In [8]:
from Loss import binary_cross_entropy,mean_binary_cross_entropy
loss,d_back = binary_cross_entropy(pred,Y_train)
print("The loss is {}\nAnd the error to be back propogated is:\n {}".format(loss , d_back))

The loss is [6.0109929]
And the error to be back propogated is:
 [[-2.        ]
 [ 1.76040923]
 [12.18658652]
 [ 9.50638367]]


In [9]:
# Now we can train the model by iteratively on each datapoint.

layer_list = [2,3, 1]
activation_list = ['sigmoid','sigmoid']
model = MultiLayerPerceptron(layer_list,activation_list)


_=model.train(X_train,
            Y_train,
            X_train,
            Y_train,
            metric ='accuracy_binary',
            loss_function_string='mean_binary_cross_entropy',
            epochs=200,
            record_at=1)

(2, 3) (3,)
<function sigmoid at 0x7fb06d3c82f0>
(3, 1) (1,)
<function sigmoid at 0x7fb06d3c82f0>
0th EPOCH:
Training Loss:[0.59932499]|Training Accuracy:0.75|Test Loss:[0.59524835]|Test Accuracy:0.75
1th EPOCH:
Training Loss:[0.59524835]|Training Accuracy:0.75|Test Loss:[0.59122565]|Test Accuracy:0.75
2th EPOCH:
Training Loss:[0.59122565]|Training Accuracy:0.75|Test Loss:[0.58725448]|Test Accuracy:0.75
3th EPOCH:
Training Loss:[0.58725448]|Training Accuracy:0.75|Test Loss:[0.58333249]|Test Accuracy:0.75
4th EPOCH:
Training Loss:[0.58333249]|Training Accuracy:0.75|Test Loss:[0.57945741]|Test Accuracy:0.75
5th EPOCH:
Training Loss:[0.57945741]|Training Accuracy:0.75|Test Loss:[0.57562707]|Test Accuracy:0.75
6th EPOCH:
Training Loss:[0.57562707]|Training Accuracy:0.75|Test Loss:[0.57183935]|Test Accuracy:0.75
7th EPOCH:
Training Loss:[0.57183935]|Training Accuracy:0.75|Test Loss:[0.56809222]|Test Accuracy:0.75
8th EPOCH:
Training Loss:[0.56809222]|Training Accuracy:0.75|Test Loss:[0.5643

In [10]:
# Now we can predict the values for unseen data or trained data also
# We can also calculate the accuracy of the model we have trained
model.metric_function(X_train,Y_train,metric='accuracy_binary')

1.0

# N Bit XOR

Now Lets try working with just a little better data. A n XOR operator. So lets create the dataset for n bit xor.

We would have atmost 2^n data point in this type of data set.BUt we would limit our dataset to a 1000 data points
whichever is smaller.

Then we can divide into training and test set

In [ ]:

n = 10
max_datapoint = 10000
datapoints = min(pow(2,n) , max_datapoint)

X = np.zeros((datapoints , n) , dtype=np.int32)
Y = np.zeros((datapoints , 1), dtype=np.int32)

for i in range(datapoints):
    tmp = i
    y_tmp = 0
    for j in range(n-1 , -1 , -1):
        X[i,j] = tmp&1
        y_tmp = y_tmp^X[i,j]
        tmp = tmp>>1
    Y[i] = y_tmp
        
        

In [ ]:
# for sanity check lets print one example
ind = 11
print(X[ind] , Y[ind])

In [ ]:
# Lets divide the set in training and testing
div = 0.9
train_n = int(div * datapoints)
X_train = X[:train_n]
Y_train = Y[:train_n]

X_test = X[train_n:]
Y_test = Y[train_n:]
Y_test.shape

In [ ]:
layer_list = [n,20,20,15,8,4,1]
activation_list = ['relu','sigmoid','sigmoid','sigmoid','tanh','sigmoid']

model = MultiLayerPerceptron(layer_list,activation_list)

In [ ]:
layer_list = [n,20,20,15,8,4,1]
activation_list = ['relu','sigmoid','sigmoid','sigmoid','tanh','sigmoid']

model = MultiLayerPerceptron(layer_list,activation_list)

In [ ]:
train_loss_his,train_acc_his,test_loss_his,test_acc_his,epoch_his = model.train(X_train,
                                                                                Y_train ,
                                                                                X_test,
                                                                                Y_test,
                                                                                accuracy_fn=accuracy,
                                                                                loss_function_string='mean_binary_cross_entropy',
                                                                                epochs = 500,
                                                                                record_at = 20,
                                                                                learning_rate= 0.1,
                                                                                learning_rate_decay=False)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
train_line, = plt.plot(epoch_his,train_loss_his,label = 'train')
test_line, = plt.plot(epoch_his,test_loss_his,label = 'test')
plt.xlabel('EPOCHS')
plt.ylabel('Loss')
plt.legend([train_line, test_line] , ['train','test'])
plt.show()

In [ ]:
train_line, = plt.plot(epoch_his,train_acc_his,label = 'train')
test_line, = plt.plot(epoch_his,test_acc_his,label = 'test')
plt.xlabel('EPOCHS')
plt.ylabel('Accuracy')
plt.legend([train_line, test_line] , ['train','test'])
plt.show()
plt.show()